In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import os

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

#print(sys.path)


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [3]:
x_train, y_train,x_test_original, y_test_original = ReadTS('RefrigerationDevices')
y_train = y_train - 1
y_test_original = y_test_original - 1
#print(x_train.shape)
#print(y_train)

In [4]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):

    original_shape = TS.shape
    
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    scaler = StandardScaler()
    
    TS_normalized = scaler.fit_transform(TS_2d)
    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train[1])

In [5]:
model = make_CNN_model(x_train.shape[1:],num_classes = 3)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=150, batch_size=32)

Epoch 1/150


2024-12-04 10:31:30.143288: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/10 [==============================] - 1s 47ms/step - loss: 0.8880 - sparse_categorical_accuracy: 0.5833 - val_loss: 1.1766 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/150
10/10 [==============================] - 0s 30ms/step - loss: 0.7610 - sparse_categorical_accuracy: 0.6600 - val_loss: 1.1900 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/150
10/10 [==============================] - 0s 32ms/step - loss: 0.7419 - sparse_categorical_accuracy: 0.6633 - val_loss: 1.1562 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/150
10/10 [==============================] - 0s 33ms/step - loss: 0.6974 - sparse_categorical_accuracy: 0.6767 - val_loss: 1.1376 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/150
10/10 [==============================] - 0s 33ms/step - loss: 0.6762 - sparse_categorical_accuracy: 0.6833 - val_loss: 1.1216 - val_sparse_categorical_accuracy: 0.0667
Epoch 6/150
10/10 [==============================] - 0s 35ms/step - loss: 0.6802 - sparse_catego

In [6]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

12/12 [==============================] - 0s 7ms/step - loss: 2.6869 - sparse_categorical_accuracy: 0.4480
Test accuracy 0.4480000138282776
Test loss 2.6868505477905273


In [7]:
def l2norm(perturb):
    difference = perturb - x_test_original
    
    squared_difference = np.square(difference)
    
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    #print(average_l2_norm)

    return average_l2_norm


def perturb_predict(strategy,index_0,index_1):
    if strategy =='RBP1fast':
        bg = multisamplebackgroundIdentification(perturb_test_set)
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1,bg = bg) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)
    else:
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)

    accuracy = accuracy_score(y_test_original, predictions_pert)
    precision = precision_score(y_test_original, predictions_pert, average='weighted')  
    recall = recall_score(y_test_original, predictions_pert, average='weighted')
    f1 = f1_score(y_test_original, predictions_pert, average='weighted')
    l2 = l2norm(x_test_pert)

    # print(strategy)
    # print(accuracy)
    # print(precision)
    # print(recall)
    # print(f1)

    return accuracy,precision,recall,f1,l2

In [8]:
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 200
index_1 = 400

strategies = ['RBP', 'RBP1fast', 'zero', 'noise', 'blur']

results = []
l2_results = []

for strategy in strategies:
    accuracy, precision, recall, f1, l2 = perturb_predict(strategy, index_0, index_1)
    results.append([strategy, accuracy, precision, recall, f1])
    l2_results.append(l2)

columns = ['Strategy', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
df_results = pd.DataFrame(results, columns=columns)
l2_columns = ['L2-Norm']
l2_results = pd.DataFrame(l2_results, columns=l2_columns)

predictions_original = model.predict(x_test_original)
predictions_original = np.argmax(predictions_original,axis=1)
accuracy = accuracy_score(y_test_original, predictions_original)
precision = precision_score(y_test_original, predictions_original, average='weighted')  
recall = recall_score(y_test_original, predictions_original, average='weighted')
f1 = f1_score(y_test_original, predictions_original, average='weighted')
original_data = ['original', accuracy, precision, recall, f1]
df_results.loc[-1] = original_data  
df_results.index = df_results.index + 1  
df_results = df_results.sort_index()  

df_results.loc[2, 'Strategy'] = 'MRBP'

current_directory = os.getcwd()
result_folder = os.path.join(current_directory, 'result_data')

csv_path = os.path.join(result_folder, 'RefrigerationDevices.csv')
df_results.to_csv(csv_path, index=False)

csv_path = os.path.join(result_folder, 'RefrigerationDevicesl2.csv')
l2_results.to_csv(csv_path, index=False)

print("done")
print(df_results)
print(l2_results)

12/12 [==============================] - 0s 7ms/step
done
   Strategy  Accuracy  Precision    Recall  F1 Score
0  original  0.448000   0.432353  0.448000  0.379870
1       RBP  0.461333   0.410094  0.461333  0.386821
2      MRBP  0.450667   0.439626  0.450667  0.425885
3      zero  0.456000   0.447477  0.456000  0.445166
4     noise  0.453333   0.406486  0.453333  0.359292
5      blur  0.480000   0.464216  0.480000  0.399490
     L2-Norm
0  10.090240
1  14.178094
2  14.069803
3  22.786790
4   3.325718
